In [1]:
import folium
import requests
import streamlit as st 
from streamlit_folium import folium_static
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely import wkt
import re

In [2]:
# Getting start/dest coordinates
def get_lat_lon(streetname):
    BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'
    response = requests.get(f'{BASE_URL}&street={streetname}&city=Stuttgart')
    data = response.json()
    
    if data:
        lat = data[0].get('lat')
        lon = data[0].get('lon')
        return float(lat), float(lon)
    else:
        # Handle the case where the geocoding service does not return valid data
        return None

In [14]:
# Initializing OSM Graph
def init_osm_graph(city):
    G = ox.graph_from_place(city, network_type='bike')
    return G

G = init_osm_graph('Stuttgart')

In [3]:
# Get fastest route from OSM
def get_osm_route(graph, start_location, dest_location, weight):
     
    start_data = get_lat_lon(start_location)
    dest_data = get_lat_lon(dest_location)

    orig_node = ox.distance.nearest_nodes(graph, start_data[1], start_data[0])
    dest_node = ox.distance.nearest_nodes(graph, dest_data[1], dest_data[0])
    shortest_route = nx.shortest_path(graph, orig_node, dest_node, weight=weight)
    pathDistance = nx.shortest_path_length(graph, orig_node, dest_node, weight=weight)
    
    return shortest_route, pathDistance

In [ ]:
shortest_route, pathDistance = get_osm_route(G, "Königstraße 4", "Tübingerstraße 40", "length")
         
m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
m

In [15]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [28]:
merged_osm = pd.read_csv('OSM\merged_osm.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2845835240.py:1: SyntaxWarning: invalid escape sequence '\m'
  merged_osm = pd.read_csv('OSM\merged_osm.csv')
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2845835240.py:1: DtypeWarning: Columns (7,8,19,20,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_osm = pd.read_csv('OSM\merged_osm.csv')


In [7]:
temp = nodes[['osmid', 'geometry']]
temp

,osmid,geometry
0,4067183,"LINESTRING (9.0915729 48.8213325, 9.0912874 48..."
1,4067183,"LINESTRING (9.0915729 48.8213325, 9.0919702 48..."
2,128522353,"LINESTRING (9.0915729 48.8213325, 9.0915312 48..."
3,8115675,"LINESTRING (9.0943289 48.8234279, 9.0941412 48..."
4,8879915,"LINESTRING (9.0943289 48.8234279, 9.0943193 48..."
...,...,...
113131,1209687037,"LINESTRING (9.178453 48.7670791, 9.1785028 48...."
113132,1209694031,"LINESTRING (9.1470322 48.727117, 9.1470401 48...."
113133,1213878443,"LINESTRING (9.1625236 48.7461401, 9.1628703 48..."
113134,1213878444,"LINESTRING (9.1628703 48.7463298, 9.1630066 48..."


In [10]:
from shapely.geometry import Point, Polygon, LineString

def extract_first_coordinates(geometry):
    try:
        if isinstance(geometry, Point):
            return geometry.x, geometry.y
        elif isinstance(geometry, LineString):
            return geometry.xy[0][0], geometry.xy[1][0]
        elif isinstance(geometry, Polygon):
            # Extract the first point of the exterior ring of the polygon
            x, y = geometry.exterior.xy[0][0], geometry.exterior.xy[1][0]
            return x, y
        else:
            return None, None
    except Exception as e:
        print(f"Error extracting coordinates: {e}")
        return None, None

# Apply the function to your DataFrame
temp[['x', 'y']] = temp['geometry'].apply(extract_first_coordinates).apply(pd.Series)


C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2153936117.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[['x', 'y']] = temp['geometry'].apply(extract_first_coordinates).apply(pd.Series)


In [11]:
temp

,osmid,geometry,x,y
0,4067183,"LINESTRING (9.0915729 48.8213325, 9.0912874 48...",None,None
1,4067183,"LINESTRING (9.0915729 48.8213325, 9.0919702 48...",None,None
2,128522353,"LINESTRING (9.0915729 48.8213325, 9.0915312 48...",None,None
3,8115675,"LINESTRING (9.0943289 48.8234279, 9.0941412 48...",None,None
4,8879915,"LINESTRING (9.0943289 48.8234279, 9.0943193 48...",None,None
...,...,...,...,...
113131,1209687037,"LINESTRING (9.178453 48.7670791, 9.1785028 48....",None,None
113132,1209694031,"LINESTRING (9.1470322 48.727117, 9.1470401 48....",None,None
113133,1213878443,"LINESTRING (9.1625236 48.7461401, 9.1628703 48...",None,None
113134,1213878444,"LINESTRING (9.1628703 48.7463298, 9.1630066 48...",None,None


In [87]:
new_nodes = pd.merge(nodes, temp, on=('osmid', 'geometry', 'x', 'y'), how='outer')
new_nodes

,osmid,y,x,street_count,highway,ref,geometry
0,433908,48.821332,9.091573,3.0,NaN,NaN,POINT (9.09157 48.82133)
1,433912,48.823428,9.094329,3.0,NaN,NaN,POINT (9.09433 48.82343)
2,433913,48.825228,9.094170,3.0,NaN,NaN,POINT (9.09417 48.82523)
3,433914,48.822942,9.092951,3.0,NaN,NaN,POINT (9.09295 48.82294)
4,433918,48.823439,9.092950,3.0,NaN,NaN,POINT (9.09295 48.82344)
...,...,...,...,...,...,...,...
149800,1209687036,48.766202,9.179195,NaN,NaN,NaN,"LINESTRING (9.17919 48.76620, 9.17914 48.76625..."
149801,1209694031,48.727117,9.147032,NaN,NaN,NaN,"LINESTRING (9.14703 48.72712, 9.14704 48.72723..."
149802,1213878443,48.746140,9.162524,NaN,NaN,NaN,"LINESTRING (9.16252 48.74614, 9.16287 48.74633)"
149803,1213878444,48.746330,9.162870,NaN,NaN,NaN,"LINESTRING (9.16287 48.74633, 9.16301 48.74640)"


In [29]:
def db_to_graph(db_nodes, db_edges):
    # nodes = gpd.GeoDataFrame(db_nodes).set_index(["osmid"])
    nodes = db_nodes
    edges = gpd.GeoDataFrame(db_edges).set_index(["u", "v", "key"])
    edges["geometry"] = edges["geometry"].astype(str).apply(lambda x: wkt.loads(x))
    edges = gpd.GeoDataFrame(edges).set_crs("epsg:4326")
    G = ox.graph_from_gdfs(nodes, edges)
    return nodes, edges, G

bike_nodes, bike_edges, G_bike = db_to_graph(nodes, merged_osm)

In [30]:
# Get fastest route from OSM
def get_bike_route(graph, start_location, dest_location, weight):
     
    start_data = get_lat_lon(start_location)
    dest_data = get_lat_lon(dest_location)

    # orig_node = ox.distance.nearest_nodes(graph, start_data[1], start_data[0])
    # dest_node = ox.distance.nearest_nodes(graph, dest_data[1], dest_data[0])
    orig_edge = ox.nearest_edges(G, start_data[1], start_data[0])
    dest_edge = ox.nearest_edges(G, dest_data[1], dest_data[0])
    best_route = ox.shortest_path(graph, orig_edge[0], dest_edge[1], weight=weight)
    # best_pathDistance = nx.shortest_path_length(graph, orig_edge, dest_edge, weight=weight)
    
    return best_route #, best_pathDistance

In [33]:
bikeable_route = get_bike_route(G_bike, "Unteraicherstrasse 22", "Schmiedgasse 5", "finalSore_reversed")
         
m = ox.plot_route_folium(G_bike, bikeable_route, tiles='openstreetmap')
m

c:\Users\Zhenzhen\AppData\Local\Programs\Python\Python312\Lib\site-packages\osmnx\routing.py:49: UserWarning: The attribute 'finalSore_reversed' is missing or null on some edges.
  _verify_edge_attribute(G, weight)
C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\2881458597.py:3: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  m = ox.plot_route_folium(G_bike, bikeable_route, tiles='openstreetmap')


In [34]:
shortest_route, pathDistance = get_osm_route(G, "Unteraicherstrasse 22", "Schmiedgasse 5", "length")
         
m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
m

C:\Users\Zhenzhen\AppData\Local\Temp\ipykernel_4704\399718455.py:3: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  m = ox.plot_route_folium(G, shortest_route, tiles='openstreetmap')
